We want $(\lambda,m^2)$ phase diagram. First we generate samples, then learn them per RG step and infer RG trajectories!

In [ ]:
using ColorSchemes

using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, Optim, GalacticOptim, Plots

# --------------------------
# Model Definitions
# --------------------------

struct Phi4Model2D
    lattice_size::Int
    lambda::Float64
    m2::Float64
    a::Float64
end

function calculate_energy_2d(phi::Matrix{T}, model::Phi4Model2D) where T<:Real
    N = size(phi, 1)
    kinetic_energy = 0.0
    potential_energy = 0.0
    a = model.a
    for i in 1:N, j in 1:N
        i_right = mod1(i + 1, N)
        j_up = mod1(j + 1, N)
        kinetic_energy += 0.5 * ((phi[i, j] - phi[i_right, j])^2 + (phi[i, j] - phi[i, j_up])^2)/a^2
        potential_energy += 0.5 * model.m2 * phi[i, j]^2 + 0.25 * model.lambda * phi[i, j]^4 
    end
    return kinetic_energy + potential_energy
end

struct Phi4TargetDensity2D
    dim::Int
    model::Phi4Model2D
end

function LogDensityProblems.logdensity(p::Phi4TargetDensity2D, phi_vector::Vector{T}) where T<:Real
    N = p.model.lattice_size
    phi = reshape(phi_vector, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::Phi4TargetDensity2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{Phi4TargetDensity2D}) = LogDensityProblems.LogDensityOrder{1}()

function generate_phi4_data_hmc_2d(model::Phi4Model2D, n_samples::Int, n_adapts::Int = 2000)
    lattice_size = model.lattice_size
    phi4_target = Phi4TargetDensity2D(lattice_size^2, model)
    initial_phi = randn(lattice_size, lattice_size)
    metric = DiagEuclideanMetric(lattice_size^2)
    hamiltonian = Hamiltonian(metric, phi4_target, ForwardDiff)
    initial_epsilon = find_good_stepsize(hamiltonian, vec(initial_phi))
    integrator = Leapfrog(initial_epsilon)
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))
    samples_vector, stats = sample(hamiltonian, kernel, vec(initial_phi), n_samples, adaptor, n_adapts; progress=false)
    samples = [reshape(sample, lattice_size, lattice_size) for sample in samples_vector]
    return samples
end

function score_matching_objective_2d(lambda::Float64, m2::Float64, a::Float64, data::Vector{Matrix{Float64}})
    N = size(data[1], 1)
    obj = 0.0
    for sample in data
        for i in 1:N, j in 1:N
            x_ij = sample[i, j]
            x_left = sample[mod1(i - 1, N), j]
            x_right = sample[mod1(i + 1, N), j]
            x_down = sample[i, mod1(j - 1, N)]
            x_up = sample[i, mod1(j + 1, N)]
            kinetic_term = -(4 * x_ij - x_left - x_right - x_up - x_down)/a^2
            score_ij = kinetic_term - (m2 * x_ij + lambda * x_ij^3)
            s_prime = -(m2 + 3 * lambda * x_ij^2 + 4/a^2)
            obj += 0.5 * score_ij^2 + s_prime
        end
    end
    return obj / length(data)
end

function estimate_parameters_score_matching_2d(data::Vector{Matrix{Float64}})
    f(x) = score_matching_objective_2d(x[1], x[2], x[3], data)
    initial_x = [0.1, 0.1, 0.8]
    result = optimize(f, initial_x, ParticleSwarm())
    x_opt = result.minimizer
    println("Estimated λ = ", x_opt[1], ", m² = ", x_opt[2], ", a = ", x_opt[3])
    return x_opt[1], x_opt[2], x_opt[3]
end

function compute_raw_grad_squared(phi::Matrix{Float64})
    gradx = circshift(phi, (-1, 0)) .- phi
    grady = circshift(phi, (0, -1)) .- phi
    return mean(gradx.^2 .+ grady.^2)
end

function block_spin_transform_with_normalization(phi::Matrix{Float64})
    nx, ny = size(phi)
    nx_new, ny_new = div(nx, 2), div(ny, 2)
    phi_blocked = zeros(Float64, nx_new, ny_new)
    for i in 1:nx_new, j in 1:ny_new
        phi_blocked[i, j] = median(vec(phi[2i-1:2i, 2j-1:2j]))
    end
    raw_grad_old = compute_raw_grad_squared(phi)
    raw_grad_new = compute_raw_grad_squared(phi_blocked)
    Z_phi = raw_grad_new / raw_grad_old
    phi_blocked_normalized = phi_blocked ./ sqrt(Z_phi)
    return phi_blocked_normalized
end

function block_spin_samples(samples::Vector{Matrix{Float64}})
    return [block_spin_transform_with_normalization(sample) for sample in samples]
end

# Run RG flow and collect (λ, m²) trajectory
function run_lambda_vs_m2_trajectory(lambda_0::Float64, m2_0::Float64, lattice_size::Int, n_samples::Int, n_steps::Int)
    model = Phi4Model2D(lattice_size, lambda_0, m2_0, 1.0)
    samples = generate_phi4_data_hmc_2d(model, n_samples)
    lambdas = [lambda_0]
    m2s = [m2_0]
    for step in 1:(n_steps - 1)
        samples = block_spin_samples(samples)
        λ, m2, _ = estimate_parameters_score_matching_2d(samples)
        push!(lambdas, λ)
        push!(m2s, m2)
    end
    return lambdas, m2s
end

function scan_and_plot_lambda_vs_m2(lambda_fixed::Float64, m2_range::Vector{Float64}, lattice_size::Int, n_samples::Int, n_steps::Int)
    plot(xlabel="m²", ylabel="λ", title="RG trajectories in (m², λ) space", legend=:topleft)
    for m2_0 in m2_range
        lambdas, m2s = run_lambda_vs_m2_trajectory(lambda_fixed, m2_0, lattice_size, n_samples, n_steps)
        color_index = findfirst(==(m2_0), m2_range)
        plot_color = get(ColorSchemes.RdYlBu_8, (color_index - 1) / (length(m2_range) - 1))
        plot!(m2s, lambdas, label="m²₀ = $m2_0", lw=2, marker=:circle, color=plot_color)

        for i in 1:length(m2s)-1
            dx = m2s[i+1] - m2s[i]
            dy = lambdas[i+1] - lambdas[i]
            quiver!([m2s[i]], [lambdas[i]], quiver=([dx], [dy]), arrow=:closed, color=plot_color, lw=1, linealpha=1.0, marker=false)
        end
    end
    savefig("rg_lambda_vs_m2.pdf")
    display(current())
end

# Run scan and plot
lattice_size = 16
lambda_0 = 0.5
m2_range = collect(-1.5:0.25:0.25)
n_samples = 20000
n_steps = 3
scan_and_plot_lambda_vs_m2(lambda_0, m2_range, lattice_size, n_samples, n_steps)


We can also scan the diagram for various values of $\lambda$ and $m^2$ to see where the critical point is for each.

In [ ]:
function scan_phase_diagram(lambda_range::Vector{Float64}, m2_range::Vector{Float64},
    lattice_size::Int, n_samples::Int, n_steps::Int)

    results = []

    for λ in lambda_range
        println("Scanning λ = $λ")
        transition_found = false

        for i in 1:length(m2_range)-1
            m2a = m2_range[i]
            m2b = m2_range[i+1]

            _, m2s_a = run_lambda_vs_m2_trajectory(λ, m2a, lattice_size, n_samples, n_steps)
            _, m2s_b = run_lambda_vs_m2_trajectory(λ, m2b, lattice_size, n_samples, n_steps)

            final_m2a = m2s_a[end]
            final_m2b = m2s_b[end]

            if sign(final_m2a - m2a) != sign(final_m2b - m2b)
                m2c_estimate = (m2a + m2b) / 2
                println("  Critical m²_c ≈ $m2c_estimate")
                push!(results, (λ = λ, mc2 = m2c_estimate))
                transition_found = true
                break
            end
        end

        if !transition_found
            println("  No transition detected in given m² range for λ = $λ")
            push!(results, (λ = λ, mc2 = NaN))
        end
    end
    return results
end


In [ ]:
lambda_range = collect(0.1:0.1:0.6)
m2_range = collect(-0.6:0.05:-0.0)
results = scan_phase_diagram(lambda_range, m2_range, 16, 15000, 3)

Code to plot the critical points in the $\lambda$ and $m^2$ phase space. We also store the samples from a previous run. Comment out the values if you need to run this separately.

In [ ]:
using LaTeXStrings, Plots, ColorSchemes
# Stored values from previous runs
mc2s = [-0.58, -0.52, -0.46, -0.40, -0.35, -0.30, -0.27, -0.25, -0.23]
λs    = [ 0.55,  0.50,  0.45,  0.40,  0.35,  0.30,  0.25,  0.20,  0.15]
# Phase boundary plot

p = plot(
    mc2s[1:6], λs[1:6],
    marker = :circle,
    lw = 2,
    xlabel = L"m^2_c",
    ylabel = L"\lambda",
    title = L"Phase Boundary $m^2_c(\lambda)$",
    palette = :RdBu_4,
    label = "Phase boundary"
)

# Symmetric phase (fill below)
plot!(
    mc2s[1:6], λs[1:6],
    fillrange = minimum(λs) .- 0.05,
    fillalpha = 0.3,
    label = "Broken symmetry phase",
    color = get(ColorSchemes.RdYlBu, 0.85),  # bluish
    lw = 0
)

# Z2 broken phase (fill above)
plot!(
    mc2s[1:6], λs[1:6],
    fillrange = maximum(λs) .+ 0.05,
    fillalpha = 0.3,
    label = "Symmetric phase",
    color = get(ColorSchemes.RdYlBu, 0.15),  # reddish
    lw = 0
)

savefig(p, "phase_bdy_lambdam^2.pdf")
display(p)
